## Hybrid Fine-tuning: OpenRouter + Local Ollama
- Data Generation: OpenRouter (high-quality training data)
- Training Target: Local Ollama model (llama2:3.1-8b-instruct)
- Hardware: RTX 3090 (24GB VRAM)

Prerequisites:
1. OpenRouter API key
2. Ollama installed
3. Run: !ollama pull llama2:3.1-8b-instruct

In [ ]:
# Install dependencies
!pip install -q accelerate==0.21.0 \
    peft==0.4.0 \
    bitsandbytes==0.40.2 \
    transformers==4.31.0 \
    trl==0.4.7 \
    openai \
    torch \
    pandas \
    datasets

In [ ]:
import os
import openai
import random

# OpenRouter Configuration
openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = "YOUR_OPENROUTER_KEY"

# Model Configuration
OPENROUTER_MODEL = "anthropic/claude-2"  # For data generation
LOCAL_MODEL = "llama2:3.1-8b-instruct"   # Local Ollama model

# Training parameters
prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
temperature = 0.4
number_of_examples = 100

In [ ]:
def generate_example(prompt, prev_examples, temperature=0.5):
    messages = [{
        "role": "system",
        "content": f"""You are generating data which will be used to train a machine learning model.
        Generate data samples with prompt/response pairs in this format:
        ```
        prompt
        -----------
        $prompt_goes_here
        -----------
        
        response
        -----------
        $response_goes_here
        -----------
        ```
        Make samples unique and diverse. Here is the model we want to train:
        {prompt}"""
    }]

    if prev_examples:
        if len(prev_examples) > 10:
            prev_examples = random.sample(prev_examples, 10)
        for example in prev_examples:
            messages.append({
                "role": "assistant", 
                "content": example
            })

    response = openai.ChatCompletion.create(
        model=OPENROUTER_MODEL,
        messages=messages,
        temperature=temperature,
        headers={"HTTP-Referer": "https://localhost:1234"}
    )
    return response.choices[0].message['content']

# Generate examples
prev_examples = []
for i in range(number_of_examples):
    print(f'Generating example {i}')
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

In [ ]:
def generate_system_message(prompt):
    messages = [{
        "role": "system",
        "content": "Generate a concise system prompt for model inference. Format: Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO"
    },
    {
        "role": "user",
        "content": prompt.strip()
    }]
    
    response = openai.ChatCompletion.create(
        model=OPENROUTER_MODEL,
        messages=messages,
        temperature=temperature,
        headers={"HTTP-Referer": "https://localhost:1234"}
    )
    return response.choices[0].message['content']

system_message = generate_system_message(prompt)
print(f'System message: {system_message}')

In [ ]:
import pandas as pd

prompts = []
responses = []

for example in prev_examples:
    try:
        split_example = example.split('-----------')
        prompts.append(split_example[1].strip())
        responses.append(split_example[3].strip())
    except:
        pass

df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})
df = df.drop_duplicates()

train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [ ]:
# RTX 3090 Optimized Training Setup
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from datasets import load_dataset

# Load datasets
train_dataset = load_dataset('json', data_files='train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='test.jsonl', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(
    lambda examples: {
        'text': [
            f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n{prompt} [/INST] {response}'
            for prompt, response in zip(examples['prompt'], examples['response'])
        ]
    },
    batched=True
)

valid_dataset_mapped = valid_dataset.map(
    lambda examples: {
        'text': [
            f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n{prompt} [/INST] {response}'
            for prompt, response in zip(examples['prompt'], examples['response'])
        ]
    },
    batched=True
)

# Load model with RTX 3090 optimizations
model = AutoModelForCausalLM.from_pretrained(
    f"ollama/{LOCAL_MODEL}",
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(f"ollama/{LOCAL_MODEL}")

# Configure LoRA for 24GB VRAM
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

# Training arguments optimized for RTX 3090
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    logging_steps=5,
    save_steps=25,
    max_grad_norm=0.3,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    gradient_checkpointing=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=2048
)

trainer.train()

In [ ]:
def generate_response(prompt):
    messages = [{
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": prompt
    }]
    
    response = ollama.chat(
        model='llama2:3.1-8b-instruct-custom',
        messages=messages,
        temperature=0.7,
        max_tokens=2048
    )
    return response['message']['content']

# Test the model
test_prompt = "Write a function that reverses a string."
result = generate_response(test_prompt)
print(result)